# Introduction to Attention

This section will cover:

* Sequence to sequence recap
* Attention overview - Encoding
* Attention overview - Decoding
* Attention encoder
* Attention decoder
* Attention encoder & decoder
* Bahdanau and Luong attention
* Multiplicative attention
* Additive attention
* Computer vision applications
* NLP application: Google neural machine translation
* Other attention methods
* The transformer and self-attention
* Lab: Attention basics

<!-- <img src="assets/images/05/img_001.png" width=700 align='center'> -->